<a href="https://colab.research.google.com/github/HofstraDoboli/TextMining_F22/blob/main/bert_sentiment_class.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install transformers
!pip install datasets

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 5.8 MB 4.7 MB/s 
     |████████████████████████████████| 182 kB 66.5 MB/s 
     |████████████████████████████████| 7.6 MB 50.9 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 451 kB 4.9 MB/s 
     |████████████████████████████████| 115 kB 82.4 MB/s 
     |████████████████████████████████| 212 kB 82.6 MB/s 
     |████████████████████████████████| 127 kB 82.0 MB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
%cd /content/drive/MyDrive/TextMining_F22/DataSets/Covid_Tweets/
%ls *.csv

/content/drive/MyDrive/TextMining_F22/DataSets/Covid_Tweets
Corona_NLP_test.csv  Corona_NLP_train.csv


In [4]:
from datasets import load_dataset
# download the data from": https://www.kaggle.com/datasets/datatattle/covid-19-nlp-text-classification 
dataset = load_dataset('csv', data_files={'train': 'Corona_NLP_train.csv', 'test': 'Corona_NLP_test.csv'}, encoding = "ISO-8859-1")

Extracting data files #0:   0%|          | 0/1 [00:00<?, ?obj/s]

Extracting data files #1:   0%|          | 0/1 [00:00<?, ?obj/s]

Generating train split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

Dataset csv downloaded and prepared to /root/.cache/huggingface/datasets/csv/default-6dd647b77931cb33/0.0.0/6b34fb8fcf56f7c8ba51dc895bfa2bfbe43546f190a60fcf74bb5e8afdcc2317. Subsequent calls will reuse this data.


  0%|          | 0/2 [00:00<?, ?it/s]

In [6]:
# documentation datasets https://huggingface.co/docs/datasets/index 
dataset


DatasetDict({
    train: Dataset({
        features: ['UserName', 'ScreenName', 'Location', 'TweetAt', 'OriginalTweet', 'Sentiment'],
        num_rows: 41157
    })
    test: Dataset({
        features: ['UserName', 'ScreenName', 'Location', 'TweetAt', 'OriginalTweet', 'Sentiment'],
        num_rows: 3798
    })
})

In [ ]:
dataset['train'][:5] # ['OriginalTweet']

{'UserName': [3799, 3800, 3801, 3802, 3803],
 'ScreenName': [48751, 48752, 48753, 48754, 48755],
 'Location': ['London', 'UK', 'Vagabonds', None, None],
 'TweetAt': ['16-03-2020',
  '16-03-2020',
  '16-03-2020',
  '16-03-2020',
  '16-03-2020'],
 'OriginalTweet': ['@MeNyrbie @Phil_Gahan @Chrisitv https://t.co/iFz9FAn2Pa and https://t.co/xX6ghGFzCC and https://t.co/I2NlzdxNo8',
  'advice Talk to your neighbours family to exchange phone numbers create contact list with phone numbers of neighbours schools employer chemist GP set up online shopping accounts if poss adequate supplies of regular meds but not over order',
  'Coronavirus Australia: Woolworths to give elderly, disabled dedicated shopping hours amid COVID-19 outbreak https://t.co/bInCA9Vp8P',
  "My food stock is not the only one which is empty...\r\r\n\r\r\nPLEASE, don't panic, THERE WILL BE ENOUGH FOOD FOR EVERYONE if you do not take more than you need. \r\r\nStay calm, stay safe.\r\r\n\r\r\n#COVID19france #COVID_19 #COVID19 #co

In [9]:
#  Tokenize data
from transformers import AutoTokenizer
import torch
device = "cuda:0" if torch.cuda.is_available() else "cpu"
PYTORCH_NO_CUDA_MEMORY_CACHING=1
# PYTORCH_CUDA_ALLOC_CONF = max_split_size_mb:128
base_model = "bert-base-cased" # distilbert-base-cased - smaller model
tokenizer = AutoTokenizer.from_pretrained(base_model) # cased = means it was trained to recognize capitalization (vs. bert-base-uncased)

Downloading:   0%|          | 0.00/29.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/570 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/213k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/436k [00:00<?, ?B/s]

In [13]:
tokenizer

PreTrainedTokenizerFast(name_or_path='bert-base-cased', vocab_size=28996, model_max_len=512, is_fast=True, padding_side='right', truncation_side='right', special_tokens={'unk_token': '[UNK]', 'sep_token': '[SEP]', 'pad_token': '[PAD]', 'cls_token': '[CLS]', 'mask_token': '[MASK]'})

In [9]:
# exaample what tokenization does
print(dataset['train'][10]["OriginalTweet"])
print(len(dataset['train'][10]["OriginalTweet"].split(' ')))
test_token = tokenizer(dataset['train'][10]["OriginalTweet"])
print(test_token.keys())
print(len(test_token['input_ids']))
print(test_token['input_ids'])


# input_ids = token's ids for parts of words in the text
# token_type_ids = all 0's if the input is one texr, 0's followed by 1's if input is a two sentence text
# attention_mask = if any of the words should be masked

All month there hasn't been crowding in the supermarkets or restaurants, however reducing all the hours and closing the malls means everyone is now using the same entrance and dependent on a single supermarket. #manila #lockdown #covid2019 #Philippines https://t.co/HxWs9LAnF9
39
dict_keys(['input_ids', 'token_type_ids', 'attention_mask'])
73
[101, 1398, 2370, 1175, 8186, 112, 189, 1151, 3515, 1158, 1107, 1103, 20247, 1116, 1137, 7724, 117, 1649, 7914, 1155, 1103, 2005, 1105, 5134, 1103, 8796, 1116, 2086, 2490, 1110, 1208, 1606, 1103, 1269, 3448, 1105, 7449, 1113, 170, 1423, 20247, 119, 108, 1299, 8009, 108, 5842, 5455, 108, 1884, 18312, 10973, 16382, 108, 4336, 18630, 131, 120, 120, 189, 119, 1884, 120, 145, 1775, 2924, 1116, 1580, 10783, 1179, 2271, 1580, 102]


In [10]:
# tokenize all dataset
def tokenize_data(example):
    return tokenizer(example['OriginalTweet'], padding='max_length', 
                     truncation = True, return_tensors = "pt").to(device)

dataset = dataset.map(tokenize_data, batched = True)  # tokenizes all the data

  0%|          | 0/42 [00:00<?, ?ba/s]

  0%|          | 0/4 [00:00<?, ?ba/s]

In [7]:
print(type(dataset['train'][1]['input_ids'])) # attention_mask is 1 for non padding tokens only, 0 for padding tokens
print(len(dataset['train'][1]['input_ids']))

<class 'list'>
512


In [11]:
# transform the labels from words to numbers
def labels2num(label):

    label = label['Sentiment']
    dict_labels = {'Positive':0, 'Negative':1, 'Neutral':2, 
                   'Extremely Positive':3, 'Extremely Negative':4}
    return {'labels': dict_labels[label]}
 
print(labels2num(dataset['train'][1]))

{'labels': 0}


In [12]:
remove_columns = ['UserName', 'ScreenName', 'Location', 'TweetAt', 'OriginalTweet', 'Sentiment']
dataset = dataset.map(labels2num, remove_columns=remove_columns)

  0%|          | 0/41157 [00:00<?, ?ex/s]

  0%|          | 0/3798 [00:00<?, ?ex/s]

In [21]:
dataset # see it added 'labels' feature and removed the rest

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'token_type_ids', 'attention_mask', 'labels'],
        num_rows: 41157
    })
    test: Dataset({
        features: ['input_ids', 'token_type_ids', 'attention_mask', 'labels'],
        num_rows: 3798
    })
})

In [23]:
len(dataset['train']['input_ids'][1:3])

2

In [13]:
# set format of the dataset to torch
dataset.set_format(type="torch", columns=["input_ids", "token_type_ids", "attention_mask", "labels"])
dataset['train'].format['type']

'torch'

In [14]:
# load training arguments, trainer, AutoModel
from transformers import AutoModelForSequenceClassification, TrainingArguments, Trainer
from transformers import DataCollatorWithPadding
#data_collator = DataCollatorWithPadding(tokenizer) # use only if you did not pad the data with the tokenizer 
# base_model = "bert-base-cased" # already defined above
model = AutoModelForSequenceClassification.from_pretrained(base_model, num_labels = 5).to(device)


Downloading:   0%|          | 0.00/436M [00:00<?, ?B/s]

Some weights of the model checkpoint at bert-base-cased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.bias', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at b

In [ ]:
model.parameters

In [15]:
training_args = TrainingArguments(
    output_dir  = 'OutputTrain', # output directory 
    num_train_epochs=3,
    evaluation_strategy = "epoch",
    learning_rate = 2e-5,
    per_device_train_batch_size = 32,
    per_device_eval_batch_size = 32,
    weight_decay = 0.01,
    push_to_hub = False,
)
# training_args


In [16]:
# set the training and validation datesets
train_dataset = dataset['train'].shuffle(seed=10).select(range(40000))
eval_dataset  = dataset['train'].shuffle(seed=10).select(range(40000, 41000)) # validation dataset

In [75]:
train_dataset[1:3]['labels'].shape # 'input_ids' 

torch.Size([2])

In [17]:
# evaluate the model 
import numpy as np
from datasets import load_metric

# metric = evaluate.combine(["accuracy", "f1", "precision", "recall"])
metric_acc    = load_metric("accuracy")
metric_f1     = load_metric("f1")
metric_prec   = load_metric("precision")
metric_recall = load_metric("recall")

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    acc    = metric_acc.compute(predictions=predictions, references=labels)
    f1     = metric_f1.compute(predictions=predictions, references=labels, average = 'macro')
    prec   = metric_prec.compute(predictions=predictions, references=labels, average = 'macro')
    recall = metric_recall.compute(predictions=predictions, references=labels, average = 'macro')
    return {"accuracy":acc, "f1": f1, "precision":prec, "recall":recall}


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:6: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  


In [18]:
trainer = Trainer(
    model= model, 
    args = training_args, 
    train_dataset= train_dataset, 
    eval_dataset = eval_dataset,
    #data_collator=data_collator, # only if you want to pad inputs during training
    compute_metrics = compute_metrics 
    )

In [20]:
# test compute_metrics 
trainer.evaluate()

***** Running Evaluation *****
  Num examples = 1000
  Batch size = 32


Trainer is attempting to log a value of "{'accuracy': 0.858}" of type <class 'dict'> for key "eval/accuracy" as a scalar. This invocation of Tensorboard's writer.add_scalar() is incorrect so we dropped this attribute.
Trainer is attempting to log a value of "{'f1': 0.863502849728043}" of type <class 'dict'> for key "eval/f1" as a scalar. This invocation of Tensorboard's writer.add_scalar() is incorrect so we dropped this attribute.
Trainer is attempting to log a value of "{'precision': 0.8619029731813951}" of type <class 'dict'> for key "eval/precision" as a scalar. This invocation of Tensorboard's writer.add_scalar() is incorrect so we dropped this attribute.
Trainer is attempting to log a value of "{'recall': 0.8665594094840111}" of type <class 'dict'> for key "eval/recall" as a scalar. This invocation of Tensorboard's writer.add_scalar() is incorrect so we dropped this attribute.


{'eval_loss': 0.4411816895008087,
 'eval_accuracy': {'accuracy': 0.858},
 'eval_f1': {'f1': 0.863502849728043},
 'eval_precision': {'precision': 0.8619029731813951},
 'eval_recall': {'recall': 0.8665594094840111},
 'eval_runtime': 7.5803,
 'eval_samples_per_second': 131.921,
 'eval_steps_per_second': 4.221,
 'epoch': 3.0}

In [15]:
!pip install pynvml
from pynvml import *

def print_gpu_utilization():
    nvmlInit()
    handle = nvmlDeviceGetHandleByIndex(0)
    info = nvmlDeviceGetMemoryInfo(handle)
    print(f"GPU memory occupied: {info.used//1024**2} MB.")


def print_summary(result):
    print(f"Time: {result.metrics['train_runtime']:.2f}")
    print(f"Samples/second: {result.metrics['train_samples_per_second']:.2f}")
    print_gpu_utilization()

print_gpu_utilization()

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
GPU memory occupied: 1321 MB.


In [19]:
trainer.train()

/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:310: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,
***** Running training *****
  Num examples = 40000
  Num Epochs = 3
  Instantaneous batch size per device = 32
  Total train batch size (w. parallel, distributed & accumulation) = 32
  Gradient Accumulation steps = 1
  Total optimization steps = 3750
  Number of trainable parameters = 108314117


Epoch,Training Loss,Validation Loss,Accuracy,F1,Precision,Recall
1,0.736300,0.569978,{'accuracy': 0.791},{'f1': 0.8000423476239357},{'precision': 0.799709557941868},{'recall': 0.8068885238065183}
2,0.434600,0.479174,{'accuracy': 0.838},{'f1': 0.8431747113676156},{'precision': 0.838936157582927},{'recall': 0.8534724735327146}
3,0.310200,0.441182,{'accuracy': 0.858},{'f1': 0.863502849728043},{'precision': 0.8619029731813951},{'recall': 0.8665594094840111}


Saving model checkpoint to OutputTrain/checkpoint-500
Configuration saved in OutputTrain/checkpoint-500/config.json
Model weights saved in OutputTrain/checkpoint-500/pytorch_model.bin
Saving model checkpoint to OutputTrain/checkpoint-1000
Configuration saved in OutputTrain/checkpoint-1000/config.json
Model weights saved in OutputTrain/checkpoint-1000/pytorch_model.bin
***** Running Evaluation *****
  Num examples = 1000
  Batch size = 32
Trainer is attempting to log a value of "{'accuracy': 0.791}" of type <class 'dict'> for key "eval/accuracy" as a scalar. This invocation of Tensorboard's writer.add_scalar() is incorrect so we dropped this attribute.
Trainer is attempting to log a value of "{'f1': 0.8000423476239357}" of type <class 'dict'> for key "eval/f1" as a scalar. This invocation of Tensorboard's writer.add_scalar() is incorrect so we dropped this attribute.
Trainer is attempting to log a value of "{'precision': 0.799709557941868}" of type <class 'dict'> for key "eval/precision

TrainOutput(global_step=3750, training_loss=0.5469261311848959, metrics={'train_runtime': 2792.2512, 'train_samples_per_second': 42.976, 'train_steps_per_second': 1.343, 'total_flos': 3.157417709568e+16, 'train_loss': 0.5469261311848959, 'epoch': 3.0})

In [22]:
trainer.train(sume_from_checkpoint = "OutputTrain/checkpoint-3500")re

Loading model from OutputTrain/checkpoint-3500.
***** Running training *****
  Num examples = 40000
  Num Epochs = 3
  Instantaneous batch size per device = 32
  Total train batch size (w. parallel, distributed & accumulation) = 32
  Gradient Accumulation steps = 1
  Total optimization steps = 3750
  Number of trainable parameters = 108314117
  Continuing training from checkpoint, will skip to saved global_step
  Continuing training from epoch 2
  Continuing training from global step 3500
  Will skip the first 2 epochs then the first 1000 batches in the first epoch. If this takes a lot of time, you can add the `--ignore_data_skip` flag to your launch command, but you will resume the training on data already seen by your model.


  0%|          | 0/1000 [00:00<?, ?it/s]

Epoch,Training Loss,Validation Loss,Accuracy,F1,Precision,Recall
3,0.310200,0.440796,{'accuracy': 0.852},{'f1': 0.857585957253941},{'precision': 0.8567195710330532},{'recall': 0.8597792422199729}


***** Running Evaluation *****
  Num examples = 1000
  Batch size = 32


Trainer is attempting to log a value of "{'accuracy': 0.852}" of type <class 'dict'> for key "eval/accuracy" as a scalar. This invocation of Tensorboard's writer.add_scalar() is incorrect so we dropped this attribute.
Trainer is attempting to log a value of "{'f1': 0.857585957253941}" of type <class 'dict'> for key "eval/f1" as a scalar. This invocation of Tensorboard's writer.add_scalar() is incorrect so we dropped this attribute.
Trainer is attempting to log a value of "{'precision': 0.8567195710330532}" of type <class 'dict'> for key "eval/precision" as a scalar. This invocation of Tensorboard's writer.add_scalar() is incorrect so we dropped this attribute.
Trainer is attempting to log a value of "{'recall': 0.8597792422199729}" of type <class 'dict'> for key "eval/recall" as a scalar. This invocation of Tensorboard's writer.add_scalar() is incorrect so we dropped this attribute.


Training completed. Do not forget to share your model on huggingface.co/models =)




TrainOutput(global_step=3750, training_loss=0.01783670450846354, metrics={'train_runtime': 200.9568, 'train_samples_per_second': 597.143, 'train_steps_per_second': 18.661, 'total_flos': 3.157417709568e+16, 'train_loss': 0.01783670450846354, 'epoch': 3.0})

In [23]:
trainer.train()

***** Running training *****
  Num examples = 40000
  Num Epochs = 3
  Instantaneous batch size per device = 32
  Total train batch size (w. parallel, distributed & accumulation) = 32
  Gradient Accumulation steps = 1
  Total optimization steps = 3750
  Number of trainable parameters = 108314117


Epoch,Training Loss,Validation Loss,Accuracy,F1,Precision,Recall
1,0.261200,0.440796,{'accuracy': 0.852},{'f1': 0.857585957253941},{'precision': 0.8567195710330532},{'recall': 0.8597792422199729}
2,0.268300,0.440796,{'accuracy': 0.852},{'f1': 0.857585957253941},{'precision': 0.8567195710330532},{'recall': 0.8597792422199729}
3,0.264100,0.440796,{'accuracy': 0.852},{'f1': 0.857585957253941},{'precision': 0.8567195710330532},{'recall': 0.8597792422199729}


Saving model checkpoint to OutputTrain/checkpoint-500
Configuration saved in OutputTrain/checkpoint-500/config.json
Model weights saved in OutputTrain/checkpoint-500/pytorch_model.bin
Saving model checkpoint to OutputTrain/checkpoint-1000
Configuration saved in OutputTrain/checkpoint-1000/config.json
Model weights saved in OutputTrain/checkpoint-1000/pytorch_model.bin
***** Running Evaluation *****
  Num examples = 1000
  Batch size = 32
Trainer is attempting to log a value of "{'accuracy': 0.852}" of type <class 'dict'> for key "eval/accuracy" as a scalar. This invocation of Tensorboard's writer.add_scalar() is incorrect so we dropped this attribute.
Trainer is attempting to log a value of "{'f1': 0.857585957253941}" of type <class 'dict'> for key "eval/f1" as a scalar. This invocation of Tensorboard's writer.add_scalar() is incorrect so we dropped this attribute.
Trainer is attempting to log a value of "{'precision': 0.8567195710330532}" of type <class 'dict'> for key "eval/precision

TrainOutput(global_step=3750, training_loss=0.26746157023111977, metrics={'train_runtime': 2776.7222, 'train_samples_per_second': 43.216, 'train_steps_per_second': 1.351, 'total_flos': 3.157417709568e+16, 'train_loss': 0.26746157023111977, 'epoch': 3.0})